In [1]:
#  1: rdd api implementation
import time
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "1") \
    .getOrCreate() \

sc = spark.sparkContext

start_time = time.time()

crime1_df= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv", \
        header=True, inferSchema=True)
crime2_df= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", \
        header=True, inferSchema=True)
crime_df= crime1_df.union(crime2_df)

mocodes_rdd = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/MO_codes.txt")
crime_data_rdd = crime_df.rdd

# take mocode , description from mocodes.txt
mocodes_map = mocodes_rdd.map(lambda x: (x.split(" ", 1)[0], x.split(" ", 1)[1]))

mocodes_counts = (
    crime_data_rdd
    .flatMap(lambda x: x[10].split() if x[10] is not None else [])     # split each string like "0913 1814 2000"
    .filter(lambda c: c.strip() != "")    # remove empty ones
    .map(lambda c: (c.strip(), 1))        # (mocode, 1)
    .reduceByKey(lambda a, b: a + b)      # count occurrences
)

joined = mocodes_counts.join(mocodes_map)
sorted_mocodes = joined.sortBy(lambda x: x[1][0], ascending=False)

end_time = time.time()

for code, (count, desc) in sorted_mocodes.take(20):
    print(f"{code} with description: {desc} has total count: {count:>6}")
print(f"RDD implementation time: {end_time-start_time:.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1114,application_1765289937462_1107,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0344 with description: Removes vict property has total count: 1002900
1822 with description: Stranger has total count: 548422
0416 with description: Hit-Hit w/ weapon has total count: 404773
0329 with description: Vandalized has total count: 377536
0913 with description: Victim knew Suspect has total count: 278618
2000 with description: Domestic violence has total count: 256188
1300 with description: Vehicle involved has total count: 219082
0400 with description: Force used has total count: 213165
1402 with description: Evidence Booked (any crime) has total count: 177470
1609 with description: Smashed has total count: 131229
1309 with description: Susp uses vehicle has total count: 122108
1202 with description: Victim was aged (60 & over) or blind/physically disabled/unable to care for self has total count: 120238
0325 with description: Took merchandise has total count: 120159
1814 with description: Susp is/was current/former boyfriend/girlfriend has total count: 118073
0444 with descr

In [1]:
# 2: Dataframe implementation
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, trim, col

import time

spark = SparkSession \
    .builder \
    .appName("dataframe query 2 execution") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "1") \
    .getOrCreate() \

sc = spark.sparkContext

start_time = time.time()

crime1_df= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv", \
        header=True, inferSchema=True)
crime2_df= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", \
        header=True, inferSchema=True)
crime_df= crime1_df.union(crime2_df)

mocodes_txt_df = spark.read.text(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/MO_codes.txt"
)
# split into (code, description)
mocodes_df = mocodes_txt_df.select(
    col("value").substr(1, 4).alias("mocode"),
    trim(col("value").substr(6, 1000)).alias("description")
)
mocode_counts_df = (
    crime_df
    .select(explode(split(col("Mocodes"), " ")).alias("mocode"))
    .filter(col("mocode") != "")     # drop empty codes
    .groupBy("mocode")
    .count()
)
# comment out the following lines if you want to test the rest of the implementations
# joined_df , choice = mocode_counts_df.join(mocodes_df, on="mocode", how="inner"),0
# joined_df.explain(True)

broadcast_join , choice = mocode_counts_df.join(mocodes_df.hint("BROADCAST"),"mocode") ,1
# merge_join , choice = mocode_counts_df.hint("MERGE").join(mocodes_df, "mocode") , 2
# sh_hash_join , choice = mocode_counts_df.hint("SHUFFLE_HASH").join(mocodes_df, "mocode") ,3 
# repnl_join , choice = mocode_counts_df.hint("SHUFFLE_REPLICATE_NL").join(mocodes_df, "mocode") ,4

# function to give the joined df from the implementation we want 
# uncomment the one we want each time and pass it to the chosen_df variable
chosen_df = broadcast_join
def collect_func(df):
    sorted_df = (df.orderBy(col("count").desc()))
    for row in sorted_df.limit(20).collect():
        print(f"{row['mocode']} with description: {row['description']} has total count: {row['count']:>6}")

collect_func(chosen_df)
end_time = time.time()
if choice==0:
    print(f"\nDataframe implementation time with catalyst optimizer: {end_time-start_time:.2f} seconds")
if choice==1:
    print(f"\nDataframe implementation time with broadcast join: {end_time-start_time:.2f} seconds")
if choice==2:
    print(f"\nDataframe implementation time with merge join: {end_time-start_time:.2f} seconds")
if choice==3:
    print(f"\nDataframe implementation time with shuffe hash join: {end_time-start_time:.2f} seconds")
if choice==4:
    print(f"\nDataframe implementation time with shuffle replicate join: {end_time-start_time:.2f} seconds")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1123,application_1765289937462_1116,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0344 with description: Removes vict property has total count: 1002900
1822 with description: Stranger has total count: 548422
0416 with description: Hit-Hit w/ weapon has total count: 404773
0329 with description: Vandalized has total count: 377536
0913 with description: Victim knew Suspect has total count: 278618
2000 with description: Domestic violence has total count: 256188
1300 with description: Vehicle involved has total count: 219082
0400 with description: Force used has total count: 213165
1402 with description: Evidence Booked (any crime) has total count: 177470
1609 with description: Smashed has total count: 131229
1309 with description: Susp uses vehicle has total count: 122108
1202 with description: Victim was aged (60 & over) or blind/physically disabled/unable to care for self has total count: 120238
0325 with description: Took merchandise has total count: 120159
1814 with description: Susp is/was current/former boyfriend/girlfriend has total count: 118073
0444 with descr

In [9]:
# import time

# def compare_join_time(df,starting_time):
#     df.count()
#     return time.time() - starting_time

# broadcast_join = mocode_counts_df.join(mocodes_df.hint("BROADCAST"), "mocode")
# merge_join = mocode_counts_df.hint("MERGE").join(mocodes_df, "mocode")
# sh_hash_join = mocode_counts_df.hint("SHUFFLE_HASH").join(mocodes_df, "mocode")
# repnl_join = mocode_counts_df.hint("SHUFFLE_REPLICATE_NL").join(mocodes_df, "mocode")

# print("Broadcast:", compare_join_time(broadcast_join))
# print("Merge:    ", compare_join_time(merge_join))
# print("ShuffleHash:", compare_join_time(sh_hash_join))
# print("ShuffleReplicateNL:", compare_join_time(repnl_join))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Broadcast: 1.6267642974853516
Merge:     1.6859371662139893
ShuffleHash: 1.4366343021392822
ShuffleReplicateNL: 1.9512789249420166

Παρατηρούμε ότι τον καλύτερο χρόνο κάνει το Shuffle hash join. Δεύτερο καλύτερο χρόνο κάνει το broadcast join. Αυτό συμβαίνει μάλλον αφού το shuffle hash join αποφεύγει την ταξινόμηση των δεδομένων ενώ το broadcast έχει χαμηλότερη απόδοση λόγο του broadcast του πίνακα mo_codes προς τους executors. Το sort merge join επιβαρύνεται από το sort ενώ το shuffle replicate nested loop πιθανώς δεν αρμόζει σε τόσο μεγάλα δεδομένα.